# How does conv2d work?

In [1]:
import tensorflow as tf

In [2]:
help(tf.nn.conv2d)

Help on function conv2d in module tensorflow.python.ops.gen_nn_ops:

conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
    Computes a 2-D convolution given 4-D `input` and `filter` tensors.
    
    Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
    and a filter / kernel tensor of shape
    `[filter_height, filter_width, in_channels, out_channels]`, this op
    performs the following:
    
    1. Flattens the filter to a 2-D matrix with shape
       `[filter_height * filter_width * in_channels, output_channels]`.
    2. Extracts image patches from the input tensor to form a *virtual*
       tensor of shape `[batch, out_height, out_width,
       filter_height * filter_width * in_channels]`.
    3. For each patch, right-multiplies the filter matrix and the image patch
       vector.
    
    In detail, with the default NHWC format,
    
        output[b, i, j, k] =
            sum_{di, dj, q} input[b, strides[1] * i + 

# Is the convolved result the sum or the mean?

I'm going to create a ones-filled 3 by 3 image and the kernel with same size.

We will try to convolve and see if the result will be 9 or 1.

if the result is 9, it means that the resulting filter is just a `sum` not the `mean`.

In [3]:
img = tf.ones([1, 3, 3, 1]) # [N, H, W, C] shape
kernel = tf.ones([3, 3, 1, 1]) # [H, W, in_C, out_C] shape
strides = [1, 1, 1, 1]

Here is the image before convolution

In [4]:
with tf.Session() as sess:
    result = sess.run(img)
result.reshape([3, 3])

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]], dtype=float32)

After convolution

In [5]:
padding = 'VALID'
conv = tf.nn.conv2d(img, kernel, strides, padding)
with tf.Session() as sess:
    result = sess.run(conv)
result.reshape([1, 1]) # notice that this is the sum not the mean

array([[ 9.]], dtype=float32)

# What values are padded with SAME padding?

After convolution again but now with `"SAME"` padding

In [6]:
padding = 'SAME'
conv = tf.nn.conv2d(img, kernel, strides, padding)
with tf.Session() as sess:
    result = sess.run(conv)
result.reshape([3, 3]) # notice that edges are padded with zeros

array([[ 4.,  6.,  4.],
       [ 6.,  9.,  6.],
       [ 4.,  6.,  4.]], dtype=float32)

# Where is the center of a kernel that has even size?
An even size kernel is 2x2, 4x4 or 6x6, etc.
We gonna find out by convolving a 2x2 image with a 2x2 kernel with SAME padding

In [7]:
img = tf.ones([1, 2, 2, 1])
kernel = tf.ones([2, 2, 1, 1])
conv = tf.nn.conv2d(img, kernel, strides, padding)
with tf.Session() as sess:
    result = sess.run(conv)
result.reshape([2, 2])

array([[ 4.,  2.],
       [ 2.,  1.]], dtype=float32)

# Conclusion
`conv2d` returns a `sum` not the `mean` like in this video: https://www.youtube.com/watch?v=FmpDIaiMIeA

`"SAME"` padding pads boundary edges with zeros and returns an image of the same size.

The center of an even size kernel is at the up-left corner.
(I'm not saying that it's the top-leftmost,
it's kinda in the center. But in an even size kernel there is no actual center so it had to choose one that is up-left)